In [1]:
import cv2
import yaml
import numpy as np
import cv2.aruco as aruco
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import csv
import os

In [3]:
def track(matrix_coefficients, distortion_coefficients):
    directory = os.path.dirname('data.csv')
    if not os.path.exists(directory):
        df = pd.DataFrame(list(), columns = ['x','y','z','alpha','beta','gamma'])
        df.to_csv('data.csv')
    while True:
        ret, frame = cap.read()
        # operations on the frame come here
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Change grayscale
        aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)  # Use 6*6 dictionary to find markers
        parameters = aruco.DetectorParameters_create()  # Marker detection parameters
# lists of ids and the corners beloning to each id
        corners, ids, rejected_img_points = aruco.detectMarkers(gray, aruco_dict,
                                                                parameters=parameters,
                                                                cameraMatrix=matrix_coefficients,
                                                                distCoeff=distortion_coefficients)
        if np.all(ids is not None):  # If there are markers found by detector
            for i in range(0, len(ids)):  # Iterate in markers
                # Estimate pose of each marker and return the values rvec and tvec---different from camera coefficients
                rvec, tvec, markerPoints = aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                           distortion_coefficients)
                (rvec - tvec).any()  # get rid of that nasty numpy value array error
                aruco.drawDetectedMarkers(frame, corners, ids)  # Draw A square around the markers
                aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.008)  # Draw Axis
                # Display the resulting frame
                cv2.putText(frame, "Distance %.1f cm Roll angle %.0f deg" % ((tvec[0][0][2] * 100 * 2.183), (rvec[0][0][1] / math.pi * 180)), (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
                df = df.append({'x':(tvec[0][0][0] * 100 * 2.183), 'y': (tvec[0][0][1] * 100 * 2.183), 'z': (tvec[0][0][2] * 100 * 2.183), 'alpha':(rvec[0][0][0] / math.pi * 180), 'beta': (rvec[0][0][1] / math.pi * 180), 'gamma': (rvec[0][0][2] / math.pi * 180) },ignore_index = True)
                #cv2.circle(image, (100, int(rr / 600)), 6, (200, 40, 230), -1)
                df.to_csv('data.csv', mode = 'a', header = False)
                R, _ = cv2.Rodrigues(rvec[0])
                cameraPose = -R.T * tvec[0]
        cv2.imshow('frame', frame)

        # Wait 3 milisecoonds for an interaction. Check the key and do the corresponding job.
        key = cv2.waitKey(3) & 0xFF
        if key == ord('q'):  # Quit
            break
    #ani = FuncAnimation(plt.gcf(), animate(df), interval=1000)
    #plt.tight_layout()
    #plt.show()
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

df.to_csv('data.csv')

In [4]:
with open(r'calibration_matrix.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    calibration_list = yaml.load(file, Loader=yaml.FullLoader)

    print(calibration_list)

{'camera_matrix': [[950.808217327545, 0.0, 621.8344954287857], [0.0, 966.6304433056954, 241.00982314842685], [0.0, 0.0, 1.0]], 'dist_coeff': [[0.03672202315425214, 0.008740819011447546, -0.040285666478112256, 0.0003312300709975179, -1.7791292452444352]]}


In [5]:
cap = cv2.VideoCapture(0)  # Get the camera source

In [6]:
mtx = np.asarray(calibration_list['camera_matrix'])
distor = np.asarray(calibration_list['dist_coeff'])

In [7]:
track(mtx, distor)